In [5]:
workspace()
import Base: copy, convert
import ForwardDiff

macro make_frame(typename, parent, fields...)
    ex = Expr(:type, true)
    push!(ex.args, Expr(:<:))
    push!(ex.args[2].args, Expr(:curly, esc(typename), :T))
    push!(ex.args[2].args, parent)
    push!(ex.args, Expr(:block))
    for field in fields
        push!(ex.args[3].args, :($(field)::T))
    end
    return ex
end


abstract Frame
function copy{T <: Frame}(frame::T)
    T([getfield(frame, x) for x in fieldnames(frame)]...)
end

function toVector(frame::Frame)
    [getfield(frame, x) for x in fieldnames(frame)]
end
convert{T,N}(T2::Type{Array{T,N}}, f::Frame) = convert(T2, toVector(f))

abstract System

num_fields(T::Type) = length(fieldnames(T))
num_states(sys::System) = num_fields(state_frame(sys))
num_inputs(sys::System) = num_fields(input_frame(sys))

@make_frame DoubleIntegratorState Frame z zdot
@make_frame DoubleIntegratorInput Frame z

type DoubleIntegrator <: System
end
state_frame(sys::DoubleIntegrator) = DoubleIntegratorState
input_frame(sys::DoubleIntegrator) = DoubleIntegratorInput

function dynamics(sys::DoubleIntegrator, t::Number, x::DoubleIntegratorState, u::DoubleIntegratorInput)
    xdot = DoubleIntegratorState(x.zdot, u.z)
end

type LinearSystem <: System
    A
    B
end

num_states(sys::LinearSystem) = size(sys.A, 2)
num_inputs(sys::LinearSystem) = size(sys.B, 2)
dynamics(sys::LinearSystem, t::Number, x::Frame, u::Frame) = typeof(x)((sys.A * toVector(x) + sys.B * toVector(u))...)

function vectorized_dynamics(sys::System, t_x_u::Vector)
    xdot = dynamics(sys, t_x_u[1], 
    state_frame(sys)(sub(t_x_u, 1 + (1:num_states(sys)))...), 
    input_frame(sys)(sub(t_x_u, 1 + num_states(sys) + (1:num_inputs(sys)))...))
    xdot
end

function linearize(sys::System, t0::Number, x0::Frame, u0::Frame)
    jac = ForwardDiff.jacobian(t_x_u::Vector -> vectorized_dynamics(sys, t_x_u))
    t_x_u = [t0; toVector(x0); toVector(u0)]
    @show t_x_u
    jac0 = jac(t_x_u)
    A = jac0[1:end, 1+(1:num_states(sys))]
    B = jac0[1:end, 1+num_states(sys)+(1:num_inputs(sys))]
    LinearSystem(A, B)
end

@make_frame PendulumState Frame theta thetadot
@make_frame PendulumInput Frame tau

type Pendulum <: System
end

function dynamics(sys::Pendulum, t::Number, x::PendulumState, u::PendulumInput)
    PendulumState(x.thetadot, u.tau - sin(x.theta) - x.thetadot)
end
state_frame(sys::Pendulum) = PendulumState
input_frame(sys::Pendulum) = PendulumInput


    /(Real, ForwardDiff.GradientNumber) at /Users/rdeits/.julia/ForwardDiff/src/GradientNumber.jl:98
is ambiguous with: 
    /(Main.ForwardDiff.GradientNumber, Real) at /Users/rdeits/.julia/ForwardDiff/src/GradientNumber.jl:105.
To fix, define 
    /(Main.ForwardDiff.GradientNumber, ForwardDiff.GradientNumber)
before the new definition.
    /(Real, ForwardDiff.GradientNumber) at /Users/rdeits/.julia/ForwardDiff/src/GradientNumber.jl:98
is ambiguous with: 
    /(Main.ForwardDiff.HessianNumber, Real) at /Users/rdeits/.julia/ForwardDiff/src/HessianNumber.jl:123.
To fix, define 
    /(Main.ForwardDiff.HessianNumber, ForwardDiff.GradientNumber)
before the new definition.
    /(Real, ForwardDiff.GradientNumber) at /Users/rdeits/.julia/ForwardDiff/src/GradientNumber.jl:98
is ambiguous with: 
    /(Main.ForwardDiff.TensorNumber, Real) at /Users/rdeits/.julia/ForwardDiff/src/TensorNumber.jl:139.
To fix, define 
    /(Main.ForwardDiff.TensorNumber, ForwardDiff.GradientNumber)
before the new defi

input_frame (generic function with 2 methods)

In [6]:
pend = Pendulum()
p_linearized = linearize(pend, 0, PendulumState(0.,0.), PendulumInput(0.))

t_x_u = [0.0,0.0,0.0,0.0]

LinearSystem(2x2 Array{Float64,2}:
  0.0   1.0
 -1.0  -1.0,2x1 Array{Float64,2}:
 0.0
 1.0)

In [10]:
@show dynamics(pend, 0, PendulumState(0.01, 0.01), PendulumInput(0.))
@show dynamics(p_linearized, 0, PendulumState(0.01, 0.01), PendulumInput(0.))

dynamics(pend,0,PendulumState(0.01,0.01),PendulumInput(0.0)) = PendulumState{Float64}(0.01,-0.019999833334166665)

PendulumState{Float64}(0.01,-0.02)


dynamics(p_linearized,0,PendulumState(0.01,0.01),PendulumInput(0.0)) = PendulumState{Float64}(0.01,-0.02)


In [7]:
sys = DoubleIntegrator()
x = DoubleIntegratorState(0.0, 1.0)
u = DoubleIntegratorInput(2.0)

xdot = copy(x)
@time for j = 1:1e3; xdot = dynamics(sys, 0, x, u); end
xdot

DoubleIntegratorState{Float64}(1.0,2.0)

In [8]:
import ForwardDiff

In [9]:
sys = DoubleIntegrator()
function do_dynamics{T}(t_x_u::Vector{T})
    t = t_x_u[1]
    x = DoubleIntegratorState(t_x_u[2:3]...)
    u = DoubleIntegratorInput(t_x_u[4])
    y = toVector(dynamics(sys, t, x, u))
end
    
do_dynamics([1, 2, 3, 4])

g = ForwardDiff.jacobian(do_dynamics)

j (generic function with 1 method)

0.002619 seconds (2.39 k allocations: 92.438 KB)


In [10]:
g(rand(4))

2x4 Array{Float64,2}:
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [11]:
@time for j = 1:1000; g([1,2,3,4]); end

  0.262861 seconds (286.91 k allocations: 13.452 MB, 1.78% gc time)
